##  Scraping des données sur lefaso.net

## Installation des librairies nécessaires
### Installation de tqdm
Utilisé pour l'animation pour l'ajout d'une barre de progression
_installation_: 


    %pip install tqdm

In [9]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Importations des bibliothèques
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import os
from urllib.parse import urljoin
from tqdm import trange

In [ ]:
BASE_LEFASO = "https://lefaso.net/spip.php"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

def _scrape_lefaso_rubrique(rubrique: int, pages: int = 10) -> pd.DataFrame:
    rows = []
    step = 20  # 20 articles par page
    for i in trange(pages, desc=f"Lefaso rub.{rubrique}"):
        offset = i * step
        params = {f"rubrique{rubrique}": "", "debut_articles": str(offset)} if i else {f"rubrique{rubrique}": ""}
        r = requests.get(BASE_LEFASO, params=params, headers=HEADERS, timeout=20)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")
        # Sur Lefaso, les titres d'articles listés sont des <h3><a ...>
        for a in soup.select("h3 a"):
            titre = a.get_text(strip=True)
            href = a.get("href")
            url = urljoin(BASE_LEFASO, href) if href else None  # Correction pour URL absolue
            rows.append({
                "source": "lefaso.net",
                "rubrique": {2: "Politique", 4: "Société"}.get(rubrique, str(rubrique)),
                "page": (i + 1),
                "titre": titre,
                "url": url,
                "date": None  # on reste simple (la date est sur la page article)
            })
        time.sleep(0.7)
    return pd.DataFrame(rows)

def get_lefaso_politique_societe(pages: int = 10) -> pd.DataFrame:
    df_pol = _scrape_lefaso_rubrique(2, pages)
    df_soc = _scrape_lefaso_rubrique(4, pages)
    return pd.concat([df_pol, df_soc], ignore_index=True)

# Pour Jupyter, pas besoin de if __name__ == "__main__"
df_lf = get_lefaso_politique_societe(10)
os.makedirs("Docs", exist_ok=True)
df_lf.to_csv("Docs/lefaso_politique_societe_p1-10.csv", index=False)
df_lf.to_json("Docs/lefaso_politique_societe_p1-10.json", orient="records", lines=True)
print(df_lf.head())

Lefaso rub.4: 100%|██████████| 10/10 [00:28<00:00,  2.85s/it]

       source   rubrique  page  \
0  lefaso.net  Politique     1   
1  lefaso.net  Politique     1   
2  lefaso.net  Politique     1   
3  lefaso.net  Politique     1   
4  lefaso.net  Politique     1   

                                               titre  \
0  Burkina : le partenariat public-privé se renfo...   
1  Burkina/ Évaluation des ministres : 54 %, la m...   
2  Burkina/Évaluation des ministres : Jacques Sos...   
3  Burkina/Evaluation des contrats d’objectifs 20...   
4  Burkina/Évaluation des ministres : Un taux glo...   

                                         url  date  
0  https://lefaso.net/spip.php?article140127  None  
1  https://lefaso.net/spip.php?article139954  None  
2  https://lefaso.net/spip.php?article139948  None  
3  https://lefaso.net/spip.php?article139947  None  
4  https://lefaso.net/spip.php?article139946  None  
